# Índices Espectrais


In [2]:
## Conectando Google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [53]:
## Importando bibliotecas
import ee
import geemap
import pandas as pd

In [6]:
## Autenticando bibliotecas do GEE
ee.Authenticate()
ee.Initialize(project = 'ee-izadorasc') # 'project' é individual para cada usuário

In [41]:
## Árae de estudo
Map = geemap.Map(center=[-7.135, -47.09], zoom=10)

region = ee.FeatureCollection("projects/ee-izadorasc/assets/limite_pncm")

Map.addLayer(region, {'color': 'black'}, 'Limite PNCM')

Map

Map(center=[-7.135, -47.09], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchData…

In [42]:
## Pontos amostrais de campo
plotPoints = ee.FeatureCollection("projects/ee-izadorasc/assets/plots_1m")

Map.addLayer(plotPoints, {'color': 'red'}, 'Pontos amostrais')

## Gerando Índices

- Definir parâmetros e funções
- Selecionar imagens
- Calcular de índices
- Exportar imagens e dataframe

In [12]:
# Funções

  ## Máscara de nuvens
def maskL8sr(image):
  qaMask = image.select('QA_PIXEL').bitwiseAnd(int('11111', 2)).eq(0)
  saturationMask = image.select('QA_RADSAT').eq(0)
  ### Fator de escala
  opticalBands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
  thermalBands = image.select('ST_B.*').multiply(0.00341802).add(149.0)

  return image.addBands(opticalBands, None, True)\
  .addBands(thermalBands, None, True).updateMask(qaMask)\
  .updateMask(saturationMask)

  ## Índices
def calc_index(image):
  ### Definindo as bandas
  # Bandas do vis (azul, verde e vermelho) do LS8: 1, 2, 3 e 4; NIR: 5;
      # SWIR 1 e 2: 6 e 7, respectivamente.
  blue = image.select('SR_B2') # Banda azul
  green = image.select('SR_B3') # Banda Verde
  red = image.select('SR_B4') # Banda Vermelha
  nir = image.select('SR_B5') # Banda NIR (Infravermelho Próximo)
  swir1 = image.select('SR_B6') # Banda SWIR1 (Infravermelho de Onda Curta 1)
  swir2 = image.select('SR_B7') # Banda SWIR2 (infravermelho de Onda Curta 2)

  ### Definindo as formulas de cada índice
  # GNDVI = (nir−green) / (nir+green)
  gndvi = image.expression('(NIR - GREEN) / (NIR + GREEN)', {
    'NIR': nir,
    'GREEN': green
  }).rename('GNDVI')

# KNDVI = tanh(((nir−red) / (nir+red)))2
  # kndvi = image.expression('tanh(((NIR - RED) / (NIR + RED)))2', {
  #   'NIR': nir,
  #   'RED': red
  # }).rename('KNDVI')

# MNDWI = (green−swir2) / (green+swir2)
  mndwi = image.normalizedDifference(['SR_B3', 'SR_B6']).rename('MNDWI')

# MSAVI = nir+0.5−(0.5∗sqrt((2∗nir+1) 2 − 8 ∗ (nir−(2∗red))))
  # msavi = image.expression('NIR + 0.5 - (0.5 * sqrt((2 * NIR + 1)**2 - 8 * (NIR - (2 * RED))))', {
  # 'NIR': nir,
  # 'RED': red}).rename('MSAVI')

# NDVI = (nir−red) / (nir+red)
  ndvi = image.normalizedDifference(['SR_B5', 'SR_B4']).rename('NDVI')

# NDWI = (green−nir) / (green+nir)
  ndwi = image.normalizedDifference(['SR_B3', 'SR_B5']).rename('NDWI')

# NDMI = (nir-swir1) / (nir+swir1)
  ndmi = image.normalizedDifference(['SR_B5', 'SR_B6']).rename('NDMI')

# SLAVI = nir / (red+swir2)
  # slavi = image.expression('NIR / (RED + SWIR2)', {
  # 'NIR': nir,
  # 'RED': red,
  # 'SWIR2': swir2}).rename('SLAVI')

# NBR = (nir−swir2) / (nir+swir2)
  nbr = image.normalizedDifference(['SR_B5', 'SR_B7']).rename('NBR')

# SAVI = (1 + L)(nir - red) / (nir + red + L)
  # savi = image.expression('(1 + 0.5)(NIR - RED) / (NIR + RED + 0.5)', {
  # 'NIR': nir,
  # 'RED': red}).rename('SAVI')

  return image.addBands([gndvi, mndwi, ndvi, ndwi, ndmi, nbr])


In [118]:
#Definindo a coleção do sensor Landsat que será utilizado
#sensor = "LANDSAT/LT05/C02/T1_L2"
#sensor = "LANDSAT/LC08/C02/T1_L2"
sensor = "LANDSAT/LC09/C02/T1_L2"
# Data de aquisição
start = '2022-06-25'
end = '2022-07-13'
# start = '2022-10-01'
# end = '2022-10-31'

# Julho = 0: Image LANDSAT/LC09/C02/T1_L2/LC09_221065_20220628 (8 bands);
        # 1: Image LANDSAT/LC09/C02/T1_L2/LC09_222065_20220705 (8 bands)
        # .filter(ee.Filter.lt('CLOUD_COVER', 10))

# Outubro = 0: Image LANDSAT/LC08/C02/T1_L2/LC08_222065_20221001 (8 bands);
          # 1: Image LANDSAT/LC08/C02/T1_L2/LC08_222065_20221017 (8 bands)
          # .filter(ee.Filter.lt('CLOUD_COVER', 50))

image = ee.ImageCollection(sensor).filterBounds(region)\
                                      .filterDate(start, end)\
                                                 .filter(ee.Filter.lt('CLOUD_COVER', 10))\
                                                 .map(maskL8sr)\
                                                 .select(['SR_B.*', 'ST_B.*'])

image

In [119]:
#Dados Mês de Julho
image = ee.ImageCollection(sensor).filterBounds(region)\
                                      .filterDate(start, end)\
                                                 .filter(ee.Filter.lt('CLOUD_COVER', 20))\
                                                 .map(maskL8sr)\
                                                 .select(['SR_B.*', 'ST_B.*'])\
                                                 .map(calc_index).median().clip(region)

## Verificando as bandas
bandas = image.bandNames()
bandas

In [120]:
#Dados Mês de Julho
imageTermal = image.select(['ST_B.*']) #selecionando somente a banda do termal
#imageTermal

## Verificando a banda
bandaTermal = imageTermal.bandNames()

bandaTermal


In [121]:
## Visualizando a área de estudo e imagem
image = image.select(image.bandNames().remove('ST_B10')) #deletando a banda termal da image

Map.addLayer(image, {'bands': ['SR_B4', 'SR_B3', 'SR_B2'], 'min': 0, 'max': 0.2},
             'Image')
Map.addLayer(imageTermal, {'bands': ['ST_B10'], 'min': 303.85, 'max': 321.47, 'palette' : ['blue', 'white', 'green']},
             'thermal')
Map

Map(bottom=136544.0, center=[-7.770865820251909, -46.45294189453125], controls=(WidgetControl(options=['positi…

In [122]:
## Padronização dos dados para valores de -1 e 1
indexImage = image.clamp(-1, 1).unitScale(-1, 1)
indexImage
# a função 'clamp' está sendo utilizada para limitar os valores de image dentro
  #do intervalo de -1 a 1, ou seja, todos os valores na image que são menores que
  #-1 serão ajustados para -1 e todos os valores que são maiores que 1 serão
  #ajustados para 1. Valores entre -1 e 1 permaneceraão inalterados.
# após o clamping a função  'unitScale'é aplicada. Essa função normaliza os
  #valores da imagem para que eles estejam em uma escala unitária entre -1 e 1.
# esta operação é comum para garantir que os dados estejam dentro de um intervalo
  #específico, sendo útil para várias aplicações, como processamento de imagens
  #ou utilização de dados em modelos de aprendizado de máquina.

In [123]:
## Obtendo dados da banda do termal
# Encontrando os valores mínimo e máximo da banda de NDVI
ndvi = image.select('NDVI')
minNDVI = ndvi.reduceRegion(ee.Reducer.min(), region, scale=30).get('NDVI').getInfo()
maxNDVI = ndvi.reduceRegion(ee.Reducer.max(), region, scale=30).get('NDVI').getInfo()

print('Minimum NDVI:', minNDVI)
print('Maximum NDVI:', maxNDVI)

Minimum NDVI: -0.40672966837882996
Maximum NDVI: 0.9136044979095459


In [124]:
# Calculando a fração da vegetação (fractional vegetation)
fv = ndvi.expression(
    '(NDVI - minNDVI) / (maxNDVI - minNDVI)', {
      'NDVI': ndvi,
      'minNDVI': minNDVI,
      'maxNDVI': maxNDVI
    }).rename('FV')

#image = image.addBands(fv)
fv

In [125]:
Map.addLayer(fv, {'bands': ['FV'], 'min': 0.11448, 'max': 0.587, 'palette' : ['blue', 'white', 'green']},
             'FV')
Map

Map(bottom=136556.0, center=[-7.100892668623642, -47.00225729968885], controls=(WidgetControl(options=['positi…

<IPython.core.display.Javascript object>

vis_params = {'bands': ['FV'], 'palette': ['#0000ff', ' #ffffff', ' #008000'], 'min': 0.4142824064422933, 'max': 0.9755899148046802, 'opacity': 1.0, 'gamma': 1.0}


In [126]:
# Calculando emissividade
em = fv.expression(
    '0.004 * FV + 0.986', {
      'FV': fv
    }).rename('EMM')

em
#image = image.addBands(em)

In [127]:
Map.addLayer(em, {'bands': ['EMM'], 'min': 0.98656, 'max': 0.989699},
             'EMM')
Map

Map(bottom=136630.0, center=[-7.201746934348412, -46.88311644173023], controls=(WidgetControl(options=['positi…

In [128]:
## Temperatura da superfície terrestre em graus Celsu (LST in Celsius Degree bring -273.15)
lst = imageTermal.expression(
    '(TB/(1+(a * (TB / b))*log(EM)))-K',{
      'TB': imageTermal.select('ST_B10'),
      'a': 0.00115,
      'b': 1.438,
      'EM': em,
      'K': 273.15}).rename('LST')
#lst

image = image.addBands(lst)
image

In [129]:
pal = ['040274', '040281', '0502a3', '0502b8', '0502ce', '0502e6',
'0602ff', '235cb1', '307ef3', '269db1', '30c8e2', '32d3ef',
'3be285', '3ff38f', '86e26f', '3ae237', 'b5e22e', 'd6e21f',
'fff705', 'ffd611', 'ffb613', 'ff8b13', 'ff6e08', 'ff500d',
'ff0000', 'de0101', 'c21301', 'a71001', '911003']

Map.addLayer(lst, {'bands': ['LST'], 'min': 27.11, 'max': 49.45, 'palette' : pal},
             'LST')
Map

Map(bottom=136630.0, center=[-7.201746934348412, -46.88311644173023], controls=(WidgetControl(options=['positi…

In [130]:
## Extraindo os valores para as bandas e amostras
extr_val = image.sampleRegions(
    collection=plotPoints,
    scale = 30,
    geometries = True,
    projection = 'EPSG:4326'
);

extr_val

In [131]:
## Gerando tabela com os dados

### Labels das bandas utilizadas
labels = ['name', 'lat', 'long', 'GNDVI', 'MNDWI', 'NDVI', 'NDWI', 'NDMI', 'NBR',
          'LST', 'SR_B1', 'SR_B2', 'SR_B3', 'SR_B4','SR_B5', 'SR_B6','SR_B7']

### Criando uma lista com os dados dos pontos amostrais
lista_df = extr_val.reduceColumns(ee.Reducer.toList(17), labels).values().get(0)

### Visualizando o dataframe (tabela dos dados) com o pandas
df = pd.DataFrame(lista_df.getInfo(), columns = labels)
df

,name,lat,long,GNDVI,MNDWI,NDVI,NDWI,NDMI,NBR,LST,SR_B1,SR_B2,SR_B3,SR_B4,SR_B5,SR_B6,SR_B7
0,WSL3P1,-7.2699,-47.0324,0.600970,-0.648023,0.531978,-0.600970,-0.077066,0.184560,38.672563,0.019230,0.026655,0.050195,0.061525,0.201390,0.235023,0.138635
1,WSL3P2,-7.2700,-47.0322,0.599291,-0.647128,0.532830,-0.599291,-0.078142,0.188699,39.524784,0.019313,0.027205,0.049177,0.059820,0.196275,0.229550,0.133960
2,WSL3P3,-7.2700,-47.0323,0.599291,-0.647128,0.532830,-0.599291,-0.078142,0.188699,39.524784,0.019313,0.027205,0.049177,0.059820,0.196275,0.229550,0.133960
3,WSL3P4,-7.2700,-47.0324,0.599291,-0.647128,0.532830,-0.599291,-0.078142,0.188699,39.524784,0.019313,0.027205,0.049177,0.059820,0.196275,0.229550,0.133960
4,OSL2P2,-7.3183,-47.0615,0.526511,-0.584248,0.432836,-0.526511,-0.083388,0.136244,44.011119,0.034107,0.044255,0.074587,0.095185,0.240468,0.284220,0.182800
5,OSL2P1,-7.3184,-47.0614,0.518350,-0.586355,0.420095,-0.518350,-0.097698,0.112754,43.807975,0.037573,0.048875,0.075330,0.096973,0.237470,0.288895,0.189345
6,OSL2P4,-7.3184,-47.0616,0.491292,-0.573289,0.383970,-0.491292,-0.114147,0.085286,43.923458,0.040130,0.052505,0.084157,0.109815,0.246710,0.310290,0.207935
7,OSL2P3,-7.3185,-47.0615,0.518350,-0.586355,0.420095,-0.518350,-0.097698,0.112754,43.807975,0.037573,0.048875,0.075330,0.096973,0.237470,0.288895,0.189345
8,OSL1P2,-7.3205,-47.0568,0.453163,-0.530367,0.344153,-0.453163,-0.101631,0.076275,45.566582,0.048050,0.065870,0.106158,0.137645,0.282103,0.345930,0.242117
9,OSL1P1,-7.3206,-47.0567,0.439319,-0.512125,0.347935,-0.439319,-0.093943,0.088963,45.084241,0.053907,0.070298,0.111328,0.138250,0.285788,0.345050,0.239092


In [132]:
## Exportando dados em formato cvs das amostras
df.to_csv('/content/drive/MyDrive/Projeto_PIBIC-CNPq/Dados/Tabelas/dados_indeces_temp_LS9_jul2022.csv')
# df.to_csv('/content/drive/MyDrive/Projeto_PIBIC-CNPq/Dados/Tabelas/dados_indeces_temp_LS8_out2022.csv')

In [133]:
## Exportando image
#'image_LS9_jul_2022_indicesTemp'
#'image_LS8_out_2022_indices'

task = ee.batch.Export.image.toDrive(
    image = image.toFloat(),
    description = 'image_LS9_jul_2022_indicesTemp',
    folder = 'Projeto_PIBIC-CNPq',
    maxPixels = 1e13,
    region = region.geometry(),
    scale = 30,
    crs='EPSG:4326',
    fileFormat = 'GeoTIFF'
)


task.start()


#https://code.earthengine.google.com/tasks

## Referências:
- Artigo: [Robust water body extraction from landsat imagery by using gradual assignment of water index and DSM](https://www.semanticscholar.org/paper/Robust-water-body-extraction-from-landsat-imagery-Puttinaovarat-Khaimook/a148721943cd58dd2ab579f0d904a8611d50419c)

- Artigo: [Mapping build-up area density using normalized difference built-up index (ndbi) and urban index (ui) wetland in the city banjarmasin - Muhaimin et al. (2022)](https://iopscience.iop.org/article/10.1088/1755-1315/1089/1/012036/pdf#:~:text=Normalized%20Difference%20Built-Up%20Index%20Method%20(NDBI)%20or%20the,calculate%20the%20built-up%20area.)

- [NDMI (Normalized Difference Moisture Index)](https://eos.com/make-an-analysis/ndmi/)

- [Modified Normalized Difference Water Index (MNDWI)](https://www.space4water.org/taxonomy/term/1246)

- Artigo: [Modification of normalised difference water index (NDWI) to enhance open water features in remotely sensed imagery](https://www.tandfonline.com/doi/abs/10.1080/01431160600589179)

- Artigo: [Utilizing NDWI, MNDWI, SAVI, WRI, and AWEI for Estimating Erosion and Deposition in Ping River in Thailand](https://www.mdpi.com/2306-5338/10/3/70)